In [1]:
import pandas as pd
import csv
import pandas_profiling
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
def remove_umlaut(s):
    u = 'ü'.encode()
    U = 'Ü'.encode()
    a = 'ä'.encode()
    A = 'Ä'.encode()
    o = 'ö'.encode()
    O = 'Ö'.encode()
    ss = 'ß'.encode()
    
    s = s.encode()
    s = s.replace(u,b'ue')
    s = s.replace(U, b'Ue')
    s = s.replace(a, b'ae')
    s = s.replace(A, b'Ae')
    s = s.replace(o, b'oe')
    s = s.replace(O, b'Oe')
    s = s.replace(ss, b'ss')

    s = s.decode('utf-8')
    return s  

In [2]:
import json
import ijson
company_authorities = []
file_entityfacts = "../../DataSets/authorities-koerperschaft_lds_20210213.jsonld"
authority_entities = ijson.items(open(file_entityfacts, 'r',encoding='utf-8'),'item')
for ind, authority_entity in enumerate(authority_entities):
    row = []
    ##########name
    value_name = ""
    if authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#preferredNameForTheCorporateBody") is not None:
        value_name = authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#preferredNameForTheCorporateBody")[0].get("@value")
    else:
        value_name = "empty_name_authorities_"+str(ind)
    row.append(value_name)
    ##########address
    value_address = "empty_address_authorities_"+str(ind)
    row.append(value_address)
    #########companyNumber
    value_companyNumber = "empty_companyNumber_authorities"+str(ind)
    row.append(value_companyNumber)
    ############CompanyYear
    value_companyYear = []
    if authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#dateOfEstablishment") is not None:
        value_companyYear = authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#dateOfEstablishment")[0].get("@value")
        if len(value_companyYear) == 4:
            value_companyYear = value_companyYear
        elif len(value_companyYear) == 10:
            value_companyYear = value_companyYear[-4:]
        elif len(value_companyYear) == 8:
            value_companyYear = value_companyYear[0:4]
           # print(value_companyYear)
    else:
        value_companyYear = "0000"
    row.append(value_companyYear)
    ############state
    value_state = "empty_state_authorities_"+str(ind)
    row.append(value_state)
    ###########status 
    value_status = "empty_status_authorities"+str(ind)
    row.append(value_status)
    ############telefon
    telefon = "0000"
    row.append(telefon)
    ############officer
    officer_name = "empty_vname empty_nachname_authotities_"+str(ind)
    position = "empty_position_authorities_"+str(ind)
    row.append(officer_name)
    row.append(position)
    ############oldAuthorities
    value_oldAuthorities = ""
    if authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#oldAuthorityNumber") is not None:
        value_oldAuthorities = authority_entity[0].get("https://d-nb.info/standards/elementset/gnd#oldAuthorityNumber")[0].get('@value')
    else:
        value_oldAuthorities = "empty_value_oldAuthorities_"+str(ind)
    row.append(value_oldAuthorities)
    #####################################
    row.append("company_authorities_id_"+str(ind))
    company_authorities.append(row)

In [3]:
company_ocdata = []
generate_id = "company_ocdata_id_"
with open("../../DataSets/de_companies_ocdata.jsonl", encoding='utf-8') as f:
    count = 0
    for line in f:
        row = []
        d = json.loads(line)
        ###################
        value_name = "" 
        value_address = ""
        value_companyNumber = "" 
        value_companyYear = ""
        value_officer = ""
        value_state = ""
        value_status = "" 
        value_officer_name = ""
        value_position = ""
        ################### iterate ele in line
        for ele,value in d.items():
            
            if ele == "all_attributes":
                #print(type(d["all_attributes"]))
                for q,v1 in d["all_attributes"].items():
                    if q == "native_company_number":
                        if v1 is not None:
                            value_companyNumber = "Amtsgericht "+v1
                            #value_companyNumber = remove_umlaut(value_companyNumber)
                        else:
                            value_companyNumber = "empty_companyNumber_ocdata"
                    elif q == "federal_state":
                        if v1 is not None:
                            value_state = v1
                            value_state = remove_umlaut(value_state)
                        else:
                            value_state = "empty_state_ocdata"
                            #print(value_state)
            elif ele == "current_status":
                if value is not None:
                    value_status = value
                    if value_status == "removed":
                        value_status = "INACTIVE"
                    else:
                        value_status = "ACTIVE"
                    #value_status = remove_umlaut(value_status)
                else:
                    value_status = "empty_status_ocdata"
            elif ele == "name":
                if value is not None and value != "" and value.isspace() != True:
                    value_name = value
                    value_name = remove_umlaut(value_name)
                else:
                    value_name = "empty_name_ocdata"
            elif ele == "registered_address":
                if value is not None:
                    value_address = value
                    value_address = remove_umlaut(value_address)
                else:
                    value_address = "empty_address_ocdata"
            elif ele == "officers":
                for key,v2 in d["officers"][0].items():
                    if key == "name":
                        if v2 is not None and v2 != "" and v2.isspace() != True:
                            value_officer_name = v2
                            value_officer_name = remove_umlaut(value_officer_name)
                        else:
                            value_officer_name = "empty_officerName_ocdata_"+str(count)
                    elif key == "position":
                        if v2 is not None:
                            value_position = v2
                            value_position = remove_umlaut(value_position)
                        else:
                            value_position = "empty_position_ocdata_"+str(count)
        row.append(value_name)
        row.append(value_address)
        row.append(value_companyNumber)
        row.append(value_companyYear)
        row.append(value_state)
        row.append(value_status)
        row.append("1000")
        row.append(value_officer_name)
        row.append("empty_position_ocdata_"+str(count))
        row.append("empty_value_ocdata_"+str(count))
        row.append(generate_id+str(count))
        company_ocdata.append(row)
        count+=1

In [4]:
matching_pairs = []
for line_authorities in company_authorities:
    for line_ocdata in company_ocdata:
        if line_authorities[0] == line_ocdata[0]:
            matching_pairs.append([line_authorities,line_ocdata])
            break
matching_pairs

KeyboardInterrupt: 

In [ ]:
non_matching_pairs = []
for line_authorities in company_authorities:
    for line_ocdata in company_ocdata:
        if line_authorities[0] != line_ocdata[0] and line_authorities[3] != line_ocdata[3]:
            non_matching_pairs.append([line_authorities,line_ocdata])
            break
    if len(non_matching_pairs) == 12:
        break
non_matching_pairs

In [ ]:
final_GS = []
for pairs in matching_pairs:
    final_GS.append([pairs[0][10],pairs[1][10],"TRUE"])
for pairs in non_matching_pairs:
    final_GS.append([pairs[0][10],pairs[1][10],"FALSE"])

In [ ]:
with open('Datasets/GS_authorities_ocdata_updated_final_version.csv', 'w', encoding = "utf-8",newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    for i in final_GS:
        filewriter.writerow(i)